# Evaluation of user pose

This notebook compare the user pose with the instructor pose and give the evaluation result

In [43]:
# imports
from utils import preprocessing

In [44]:
import importlib
importlib.reload(preprocessing)

<module 'utils.preprocessing' from '/home/deepak/projects/stanford/cs230project/baseline/utils/preprocessing.py'>

In [45]:
#what we need 
def YogaEvaluation(path_to_video: str):
    # steps:
    # process the video
    # get classification
    # evaluate
    
    # pre processing
    return preprocessing.preprocess(path_to_video)
    

In [72]:
# preprocess the instructor pose if not already done 
df_ins = YogaEvaluation('Yoga_Vid_Collected/Trik/Abhay_Trik.mp4')

# preprocess the user pose to create the angle timeseries
df_user = YogaEvaluation('deepakvid/deepakwronglast1.mp4')

Yoga_Vid_Collected/Trik/Abhay_Trik.mp4


In [73]:
# store the time series
df_ins.to_csv('project_code/utils/ins_pose/4.csv')
df_user.to_csv('deepakvid/deepakposeangle.csv')

In [74]:
df_user

,leftWES,leftESH,leftSHK,leftHKA,rightWES,rightESH,rightSHK,rightHKA
0,172.787562,29.435308,171.065279,177.415243,179.614851,19.104916,156.986766,179.937839
1,171.216386,36.498787,172.273469,179.928194,178.265628,18.528290,155.877717,179.450626
2,173.810484,41.467166,172.134579,178.129718,178.216132,18.548080,156.562058,179.395574
3,175.884705,43.607180,173.059197,178.514873,175.639188,17.846903,155.741839,179.135870
4,175.790899,43.788663,172.749099,178.232499,176.882930,18.410214,155.325997,179.682907
...,...,...,...,...,...,...,...,...
1327,174.986610,113.529277,179.670771,178.777232,176.060262,35.154686,132.740549,172.927118
1328,175.948158,114.634474,179.791908,179.013906,176.079102,35.719442,132.558590,173.022452
1329,174.274883,108.742655,179.880764,179.204588,174.249078,34.574618,134.947888,174.525756
1330,170.318464,94.752140,179.337127,179.465555,173.965895,30.687030,138.853827,175.682150


### DTW (Dynamic Time Warping)
Using DTW the distance between the two temporal sequences(user angle and instructor angle) calculated.
For every point in user pose time series the point with highest resemblence is found in the instructor series while keeping the sequence behaviour intact

In [49]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

angles = ['leftWES','leftESH','leftSHK','leftHKA','rightWES','rightESH','rightSHK','rightHKA']
angle_dtw_dict = {}
length = len(df_user['leftWES'].values)
angle_diff_dict = {'leftWES':[0]*length,'leftESH':[0]*length,'leftSHK':[0]*length,'leftHKA':[0]*length,'rightWES':[0]*length,'rightESH':[0]*length,'rightSHK':[0]*length,'rightHKA':[0]*length}
for i in angles:
    ls=list(df_user[i].values)
    ls2=list(df_ins[i].values)
    distance, path = fastdtw(ls, ls2)
    angle_dtw_dict[i]={'distance':distance,'path':path}
    
    for n,t in enumerate(angle_dtw_dict[i]['path']):
        angle_diff_dict[i][t[0]]=(abs(ls[t[0]]-ls2[t[1]]))
        

#print(angle_diff_dict)

In [50]:
angle_diff_dict.keys()
keys = ['leftWES', 'leftESH', 'leftSHK', 'leftHKA', 'rightWES', 'rightESH', 'rightSHK', 'rightHKA']
for i in keys:
    print(len(angle_diff_dict[i]))

506
506
506
506
506
506
506
506


In [51]:
import pandas as pd
angle_diff_df = pd.DataFrame(angle_diff_dict)


In [52]:
angle_diff_df

,leftWES,leftESH,leftSHK,leftHKA,rightWES,rightESH,rightSHK,rightHKA
0,6.462160,19.822037,0.861499,0.022800,0.362616,6.074393,14.863260,3.959195
1,6.848475,27.257523,0.531530,0.255340,0.173226,5.182848,14.991361,3.061749
2,0.187369,32.099521,0.571598,0.083438,2.165086,4.893288,15.491416,2.257669
3,0.583606,34.172095,0.136973,0.213297,1.360814,5.259271,15.692656,1.680686
4,1.112508,34.687050,0.059164,0.626713,0.073536,7.297526,15.640424,2.410000
...,...,...,...,...,...,...,...,...
501,2.944440,105.158412,0.481506,0.110056,0.043182,18.886344,32.627298,0.596921
502,2.153634,105.931210,1.827716,0.537362,0.048807,18.973214,32.606429,0.621244
503,4.914671,99.565256,3.033087,0.142979,0.152495,19.001162,32.919684,0.646959
504,8.664167,85.706529,3.870762,0.779828,4.298857,18.981260,32.648379,0.359444


Based on the difference the each angle is assigned a color

In [53]:
def angleColor(angle):
    if angle<=25:
        return (0,255,0)
    elif angle<=50:
        return (0,0,255)
    else:
        return (255,0,0)
    
angle_diff_df_copy = angle_diff_df
angle_diff_df_copy=angle_diff_df_copy.applymap(angleColor)

In [11]:
# df_user = pd.read_csv("yoga3.csv")
# df_user.iloc[0].values

array([172.76899073,  25.43256478, 120.07488718,  31.46674636,
       169.03535451,  25.5609926 , 115.4719288 ,  33.6625747 ])

In [2]:
angle_diff_df=pd.read_csv('deepakvid/deepakposeanglediff.csv')

## Evaluation
Using blazepose again to get the coordinate a sekeleton is drawn on user video.

The color of the skeleton are based on the difference of the angle (evaluated earlier)

The final video is displayed as the output frame by frame and is also stored to create a mp4 later.

In [54]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculateAngle(x):
    a = np.array(x[0])
    b = np.array(x[1])
    c = np.array(x[2])

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return int(angle)


# https://media.giphy.com/media/giX67VJcszXgP2RhcV/giphy.mp4
# https://media.giphy.com/media/U1elvRvDJjGKf9P1e5/giphy.mp4
cap = cv2.VideoCapture('deepakvid/deepakwronglast1.mp4')
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

df_user = pd.read_csv('deepakvid/deepakposeangle.csv')
df_angle = angle_diff_df
df_color = angle_diff_df_copy
x = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    i = 0
    while cap.isOpened():
        df = list(df_user.iloc[x].values)
        angle_name_lst = df_angle.iloc[x]
        line_color_lst = df_color.iloc[x]

        ret, frame = cap.read()

        if not ret:
            break

        # Recolor image to RGB
        image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image1.flags.writeable = False

        # Make detection
        results = pose.process(image1)

        # Recolor back to BGR
        image1.flags.writeable = True
        image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            # Get coordinates
            landmarks = results.pose_landmarks.landmark
            setOfJoints = mp_pose.PoseLandmark

            leftWrist = [landmarks[setOfJoints.LEFT_WRIST].x, landmarks[setOfJoints.LEFT_WRIST].y]
            leftElbow = [landmarks[setOfJoints.LEFT_ELBOW].x, landmarks[setOfJoints.LEFT_ELBOW].y]
            leftShoulder = [landmarks[setOfJoints.LEFT_SHOULDER].x, landmarks[setOfJoints.LEFT_SHOULDER].y]
            leftHip = [landmarks[setOfJoints.LEFT_HIP].x, landmarks[setOfJoints.LEFT_HIP].y]
            leftKnee = [landmarks[setOfJoints.LEFT_KNEE].x, landmarks[setOfJoints.LEFT_KNEE].y]
            leftAnkle = [landmarks[setOfJoints.LEFT_ANKLE].x, landmarks[setOfJoints.LEFT_ANKLE].y]
            leftEye = [landmarks[setOfJoints.LEFT_EYE].x, landmarks[setOfJoints.LEFT_EYE].y]
            leftEyeinner = [landmarks[setOfJoints.LEFT_EYE_INNER].x, landmarks[setOfJoints.LEFT_EYE_INNER].y]
            leftEar = [landmarks[setOfJoints.LEFT_EAR].x, landmarks[setOfJoints.LEFT_EAR].y]
            leftPinky = [landmarks[setOfJoints.LEFT_PINKY].x, landmarks[setOfJoints.LEFT_PINKY].y]
            leftThumb = [landmarks[setOfJoints.LEFT_THUMB].x, landmarks[setOfJoints.LEFT_THUMB].y]
            leftHeel = [landmarks[setOfJoints.LEFT_HEEL].x, landmarks[setOfJoints.LEFT_HEEL].y]
            leftFootindex = [landmarks[setOfJoints.LEFT_FOOT_INDEX].x, landmarks[setOfJoints.LEFT_FOOT_INDEX].y]

            nose = [landmarks[setOfJoints.NOSE].x, landmarks[setOfJoints.NOSE].y]

            rightWrist = [landmarks[setOfJoints.RIGHT_WRIST].x, landmarks[setOfJoints.RIGHT_WRIST].y]
            rightElbow = [landmarks[setOfJoints.RIGHT_ELBOW].x, landmarks[setOfJoints.RIGHT_ELBOW].y]
            rightShoulder = [landmarks[setOfJoints.RIGHT_SHOULDER].x, landmarks[setOfJoints.RIGHT_SHOULDER].y]
            rightHip = [landmarks[setOfJoints.RIGHT_HIP].x, landmarks[setOfJoints.RIGHT_HIP].y]
            rightKnee = [landmarks[setOfJoints.RIGHT_KNEE].x, landmarks[setOfJoints.RIGHT_KNEE].y]
            rightAnkle = [landmarks[setOfJoints.RIGHT_ANKLE].x, landmarks[setOfJoints.RIGHT_ANKLE].y]
            rightEye = [landmarks[setOfJoints.RIGHT_EYE].x, landmarks[setOfJoints.RIGHT_EYE].y]
            rightEyeinner = [landmarks[setOfJoints.RIGHT_EYE_INNER].x, landmarks[setOfJoints.RIGHT_EYE_INNER].y]
            rightEar = [landmarks[setOfJoints.RIGHT_EAR].x, landmarks[setOfJoints.RIGHT_EAR].y]
            rightPinky = [landmarks[setOfJoints.RIGHT_PINKY].x, landmarks[setOfJoints.RIGHT_PINKY].y]
            rightThumb = [landmarks[setOfJoints.RIGHT_THUMB].x, landmarks[setOfJoints.RIGHT_THUMB].y]
            rightHeel = [landmarks[setOfJoints.RIGHT_HEEL].x, landmarks[setOfJoints.RIGHT_HEEL].y]
            rightFootindex = [landmarks[setOfJoints.RIGHT_FOOT_INDEX].x, landmarks[setOfJoints.RIGHT_FOOT_INDEX].y]

            angleDict = {'leftWES': (leftWrist, leftElbow, leftShoulder),
                         'leftESH': (leftElbow, leftShoulder, leftHip),
                         'leftSHK': (leftShoulder, leftHip, leftKnee),
                         'leftHKA': (leftHip, leftKnee, leftAnkle),
                         'rightWES': (rightWrist, rightElbow, rightShoulder),
                         'rightESH': (rightElbow, rightShoulder, rightHip),
                         'rightSHK': (rightShoulder, rightHip, rightKnee),
                         'rightHKA': (rightHip, rightKnee, rightAnkle)}

            # Blank Frame for Skeleton
            image = np.zeros((int(height), int(width), 3), dtype='uint8')
            #image.fill(255)

            # Orange : (255, 69, 0)
            # Red : (255,0,0)

            # Visualize angle
            # 1.Elbow joint
            # Left Elbow
            # 'leftWES': (leftWrist, leftElbow, leftShoulder),
            

            cv2.putText(image, str(angle_name_lst['leftWES']),
                        tuple(np.multiply(leftElbow, [width+20, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(leftShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(leftElbow, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(leftWrist, [width, height]).astype(int)),
                     tuple(np.multiply(leftElbow, [width, height]).astype(int)), line_color_lst['leftWES'], thickness=5)

            # Right Elbow
            # 'rightWES': (rightWrist, rightElbow, rightShoulder)


            cv2.putText(image, str(angle_name_lst['rightWES']),
                        tuple(np.multiply(rightElbow, [width-330, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(rightShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(rightElbow, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(rightWrist, [width, height]).astype(int)),
                     tuple(np.multiply(rightElbow, [width, height]).astype(int)), line_color_lst['rightWES'], thickness=5)

            # 2.Shoulder joint
            # Left Shoulder
            # 'leftESH': (leftElbow, leftShoulder, leftHip),

            cv2.putText(image, str(angle_name_lst['leftESH']),
                        tuple(np.multiply(leftShoulder, [width+20, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(leftShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(leftElbow, [width, height]).astype(int)), line_color_lst['leftESH'], thickness=5)
            cv2.line(image, tuple(np.multiply(leftHip, [width, height]).astype(int)),
                     tuple(np.multiply(leftShoulder, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Right Shoulder
            # 'rightESH': (rightElbow, rightShoulder, rightHip),

            cv2.putText(image, str(angle_name_lst['rightESH']),
                        tuple(np.multiply(rightShoulder, [width-350, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(rightShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(rightElbow, [width, height]).astype(int)),line_color_lst['rightESH'], thickness=5)
            cv2.line(image, tuple(np.multiply(rightHip, [width, height]).astype(int)),
                     tuple(np.multiply(rightShoulder, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # 3.Hip joint
            # Left Hip
            # 'leftSHK': (leftShoulder, leftHip, leftKnee),

            cv2.putText(image, str(angle_name_lst['leftSHK']),
                        tuple(np.multiply(leftHip, [width+20, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(leftShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(leftHip, [width, height]).astype(int)),line_color_lst['leftSHK'], thickness=5)
            cv2.line(image, tuple(np.multiply(leftHip, [width, height]).astype(int)),
                     tuple(np.multiply(leftKnee, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Right Hip
            # 'rightSHK': (rightShoulder, rightHip, rightKnee),

            cv2.putText(image, str(angle_name_lst['rightSHK']),
                        tuple(np.multiply(rightHip, [width-320, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(rightShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(rightHip, [width, height]).astype(int)), line_color_lst['rightSHK'], thickness=5)
            cv2.line(image, tuple(np.multiply(rightHip, [width, height]).astype(int)),
                     tuple(np.multiply(rightKnee, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # 4.Knee joint
            # Left Knee
            # 'leftHKA': (leftHip, leftKnee, leftAnkle),
            cv2.putText(image, str(angle_name_lst['leftHKA']),
                        tuple(np.multiply(leftKnee, [width+20, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(leftKnee, [width, height]).astype(int)),
                     tuple(np.multiply(leftHip, [width, height]).astype(int)), line_color_lst['leftHKA'], thickness=5)
            cv2.line(image, tuple(np.multiply(leftKnee, [width, height]).astype(int)),
                     tuple(np.multiply(leftAnkle, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Right Knee
            # 'rightHKA': (rightHip, rightKnee, rightAnkle),

            cv2.putText(image, str(angle_name_lst['rightHKA']),
                        tuple(np.multiply(rightKnee, [width-420, height]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.line(image, tuple(np.multiply(rightKnee, [width, height]).astype(int)),
                     tuple(np.multiply(rightHip, [width, height]).astype(int)), line_color_lst['rightHKA'], thickness=5)
            cv2.line(image, tuple(np.multiply(rightAnkle, [width, height]).astype(int)),
                     tuple(np.multiply(rightKnee, [width, height]).astype(int)), (0, 255, 0), thickness=5)


            # connecting wrist to index and thumb
            # right
            cv2.line(image, tuple(np.multiply(rightWrist, [width, height]).astype(int)),
                     tuple(np.multiply(rightPinky, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(rightWrist, [width, height]).astype(int)),
                     tuple(np.multiply(rightThumb, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            # left
            cv2.line(image, tuple(np.multiply(leftWrist, [width, height]).astype(int)),
                     tuple(np.multiply(leftPinky, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(leftWrist, [width, height]).astype(int)),
                     tuple(np.multiply(leftThumb, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # connecting ankle to index and heel
            # right
            cv2.line(image, tuple(np.multiply(rightAnkle, [width, height]).astype(int)),
                     tuple(np.multiply(rightFootindex, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(rightAnkle, [width, height]).astype(int)),
                     tuple(np.multiply(rightHeel, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            # left
            cv2.line(image, tuple(np.multiply(leftAnkle, [width, height]).astype(int)),
                     tuple(np.multiply(leftFootindex, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(leftAnkle, [width, height]).astype(int)),
                     tuple(np.multiply(leftFootindex, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting Shoulders
            cv2.line(image, tuple(np.multiply(rightShoulder, [width, height]).astype(int)),
                     tuple(np.multiply(leftShoulder, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Shoulder connection midpoint calculation
            x1, y1 = tuple(np.multiply(rightShoulder, [width, height]).astype(int))
            x2, y2 = tuple(np.multiply(leftShoulder, [width, height]).astype(int))
            x_m_point = (x1 + x2) // 2
            y_m_point = (y1 + y2) // 2

            # connecting nose and shoulder midpoint
            cv2.line(image, (x_m_point, y_m_point),
                     tuple(np.multiply(nose, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting Hip
            cv2.line(image, tuple(np.multiply(rightHip, [width, height]).astype(int)),
                     tuple(np.multiply(leftHip, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting right eye to nose
            cv2.line(image, tuple(np.multiply(rightEyeinner, [width, height]).astype(int)),
                     tuple(np.multiply(rightEye, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(rightEyeinner, [width, height]).astype(int)),
                     tuple(np.multiply(nose, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting left eye to nose
            cv2.line(image, tuple(np.multiply(leftEyeinner, [width, height]).astype(int)),
                     tuple(np.multiply(leftEye, [width, height]).astype(int)), (0, 255, 0), thickness=5)
            cv2.line(image, tuple(np.multiply(leftEyeinner, [width, height]).astype(int)),
                     tuple(np.multiply(nose, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting left eye to left ear
            cv2.line(image, tuple(np.multiply(leftEye, [width, height]).astype(int)),
                     tuple(np.multiply(leftEar, [width, height]).astype(int)), (0, 255, 0), thickness=5)

            # Connecting right eye to right ear
            cv2.line(image, tuple(np.multiply(rightEye, [width, height]).astype(int)),
                     tuple(np.multiply(rightEar, [width, height]).astype(int)), (0, 255, 0), thickness=5)

        except:
            pass

        # Display User Feed
        cv2.imshow('Saved User Feed', image1)
        # Display Skeleton Frame
        cv2.imshow('Skeleton', image)
        # Capture the frames
        cv2.imwrite(f'{"video/"}frame{i}.jpg', image)
        # overlay
        tr = 0.3 # transparency between 0-1, show camera if 0
        frame = ((1-tr) * image1.astype(np.float) + tr * image.astype(np.float)).astype(np.uint8)
        #out.write(cv2.flip(frame,0))
        cv2.imshow('Transparent result', frame)
        cv2.imwrite(f'{"deepakvid/vid/"}frame{i}.png', frame)
        i = i + 1

        x = x + 1

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

/tmp/ipykernel_299686/2590622172.py:278: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frame = ((1-tr) * image1.astype(np.float) + tr * image.astype(np.float)).astype(np.uint8)


The frames stored in the above code can be used to create a video as follows:

In [55]:
import cv2
import numpy as np

img=[]
for i in range(0,500):
    img.append(cv2.imread('deepakvid/vid/frame'+str(i)+'.png'))

height,width,layers=img[1].shape
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video=cv2.VideoWriter('videoyoga.mp4',fourcc,20,(width,height))

for j in range(0,500):
    video.write(img[j])

cv2.destroyAllWindows()
video.release()

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


## Code to take input from camera

In [26]:
# not required
import cv2
import time
import numpy as np

current_milli_time = lambda: int(round(time.time() * 1000))

# Camera feed
cap_cam = cv2.VideoCapture(0)
if not cap_cam.isOpened():
    print('Cannot open camera')
    exit()
ret, frame_cam = cap_cam.read()
if not ret:
    print('Cannot open camera stream')
    cap_cam.release()
    exit()

# Video feed
filename = 'yoga_min.mp4'
cap_vid = cv2.VideoCapture(filename)
if not cap_cam.isOpened():
    print('Cannot open video: ' + filename)
    cap_cam.release()
    exit()
ret, frame_vid = cap_vid.read()
if not ret:
    print('Cannot open video stream: ' + filename)
    cap_cam.release()
    cap_vid.release()
    exit()

# Specify maximum video time in milliseconds
max_time = 1000 * cap_vid.get(cv2.CAP_PROP_FRAME_COUNT) / cap_vid.get(cv2.CAP_PROP_FPS)

# Resize the camera frame to the size of the video
height = int(cap_vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap_vid.get(cv2.CAP_PROP_FRAME_WIDTH))

# Starting from now, syncronize the videos
start = current_milli_time()

while True:
    # Capture the next frame from camera
    ret, frame_cam = cap_cam.read()
    if not ret:
        print('Cannot receive frame from camera')
        break
    frame_cam = cv2.resize(frame_cam, (width, height), interpolation = cv2.INTER_AREA)

    # Capture the frame at the current time point
    time_passed = current_milli_time() - start
    if time_passed > max_time:
        print('Video time exceeded. Quitting...')
        break
    ret = cap_vid.set(cv2.CAP_PROP_POS_MSEC, time_passed)
    if not ret:
        print('An error occured while setting video time')
        break
    ret, frame_vid = cap_vid.read()
    if not ret:
        print('Cannot read from video stream')
        break

    # Blend the two images and show the result
    tr = 0.3 # transparency between 0-1, show camera if 0
    frame = ((1-tr) * frame_cam.astype(np.float) + tr * frame_vid.astype(np.float)).astype(np.uint8)
    cv2.imshow('Transparent result', frame)
    if cv2.waitKey(1) == 27: # ESC is pressed
        break

cap_cam.release()
cap_vid.release()
cv2.destroyAllWindows()
# # Videos
# capture = cv.VideoCapture('yoga_min.mp4')
#
# while True:
#    isTrue, frame = capture.read()
#
#    blur = cv.GaussianBlur(frame, (5,5), cv.BORDER_DEFAULT)
#
#    cany = cv.Canny(blur, 125, 175)
#    cv.imshow('Cany Edges', cany)
#
#    ret, thresh = cv.threshold(cany, 125, 255, cv.THRESH_BINARY)
#    cv.imshow('Thresh', thresh)
#
#    contours, hierarchies = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
#
#    print(len(contours))
#
#    if cv.waitKey(20) & 0xFF == ord('d'):
#        break
#
#
# capture.release()
# cv.destroyAllWindows()

Cannot open camera
Cannot open camera stream
Cannot open video: yoga_min.mp4
Cannot receive frame from camera


[ WARN:0@498.867] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index
